In [1]:
using Oscar
using Combinatorics
n_elements = 7;
rk = 3;
db = Polymake.Polydb.get_db();
collection = db["Matroids.Small"];
query = Dict("RANK"=>rk,"N_ELEMENTS"=>n_elements);
results = Polymake.Polydb.find(collection, query);
oscar_matroids = [Matroid(pm) for pm in results];


  ___   ____   ____    _    ____
 / _ \ / ___| / ___|  / \  |  _ \   |  Combining ANTIC, GAP, Polymake, Singular
| | | |\___ \| |     / _ \ | |_) |  |  Type "?Oscar" for more information
| |_| | ___) | |___ / ___ \|  _ <   |  Manual: https://docs.oscar-system.org
 \___/ |____/ \____/_/   \_\_| \_\  |  Version 1.3.1


In [2]:
all_matroids =Matroid[];
counter =0
ground_set = [i for i in 1:n_elements];
permuts = permutations(ground_set);
for m in oscar_matroids
    iso= [];
    for perm in permuts
        #println("current permutation is")
        #println(perm)
        current_b = Set();
        #println(current_b);

        for b in bases(m)
            #println("current base is" )
            #println(b)
            permuted_b = Set()
            for e in b
                push!(permuted_b, perm[e])
            end
            push!(current_b, permuted_b);
        end
        current_b = Set([Set(b) for b in current_b])
    if !(current_b in iso)
        push!(iso, current_b);
        counter = counter +1;
            #println(iso)
            #println(iso);
    end
    end 
    for m in iso
        push!(all_matroids, (matroid_from_bases(m, n_elements)))      
    end
   
end

dummy = matroid_from_bases([[]], n_elements)
# insert a dummy to make indices of subs and all_matroids fit. The dummy in subs2 is needed to have an artificial minimum
insert!(all_matroids, 1, dummy)
#details: add 
N = length(all_matroids)
println(N)


33443


In [ ]:
# check if a poset admits a rank function

# The poset p is encoded as a vector of elements that can be compared with <
# p must have a least element, stored as p[1]

# true if y covers x in p
covers(p, y, x) = x < y && !any(z -> x < z < y, p)

# the length of some maximal chain in p from p[1] to x
getrank(p, x) = x == p[1] ? 0 : getdist(p, p[1], x)

# the length of some maximal chain in p from x to y where x < y
function getdist(p, x, y)
    i = findfirst(z -> x < z < y, p)
    i === nothing ? 1 : getdist(p, x, p[i]) + getdist(p, p[i], y)
end

# true if p has a rank function
function hasrank(p)
    r = map(x -> getrank(p, x), p)
    for (i, x) in enumerate(p), (j, y) in enumerate(p)
         !covers(p, y, x) || r[j] == r[i]+1 || return [i, j, r[i], r[j], covers(p, y, x)]
    end
    true
end

# example: the poset of all subsets of some set

using Random: randperm
using SmallCollections: SmallBitSet, subsets

const S = SmallBitSet{UInt32}

N = 11
q = subsets(N) # all subsets of 1:N in colexicographic ordering
r = randperm(length(q))
println(q[3])
# random permutation of all subsets of 1:N, but with the empty set as first element
p = S[i == 1 || r[i] == 1 ? q[i] : q[r[i]] for i in 1:length(q)]
# for sets, `<` means proper subset

hasrank(p)  # true


SmallBitSet{UInt64}([2])


true

In [ ]:
using Polymake
p = polytope.Polytope(POINTS = [1 1 0 0 0 1 0; 0 1 1 0 1 0 0; 0 1 0 1 0 0 1; 0 1 1 0 0 1 0; 0 0 1 1 0 1 0; 0 1 1 0 0 0 1; 1 0 1 0 0 1 0; 1 0 0 0 1 1 0; 0 1 0 0 0 1 1; 0 0 0 1 1 0 1; 0 1 1 1 0 0 0; 1 0 1 1 0 0 0; 0 0 0 1 1 1 0; 1 0 1 0 0 0 1; 1 0 1 0 1 0 0; 1 0 0 1 0 0 1; 0 0 1 0 0 1 1; 1 1 0 0 1 0 0; 1 1 0 0 0 0 1; 1 0 0 0 0 1 1; 1 1 0 1 0 0 0; 0 0 0 0 1 1 1; 0 1 0 1 1 0 0; 0 1 0 0 1 1 0; 0 0 0 1 0 1 1; 1 0 0 1 0 1 0; 1 0 0 0 1 0 1; 0 0 1 1 1 0 0; 0 0 1 0 1 0 1
])
T = p.VERTICES_IN_FACETS;
q = polytope.Polytope(POINTS = [1 1 0 0 0 1 0; 0 1 1 0 1 0 0; 0 1 0 1 0 0 1; 0 1 1 0 0 1 0; 0 0 1 1 0 1 0; 0 1 1 0 0 0 1; 1 0 1 0 0 1 0; 1 0 0 0 1 1 0; 0 1 0 0 0 1 1; 0 0 0 1 1 0 1; 0 1 1 1 0 0 0; 1 0 1 1 0 0 0; 0 0 0 1 1 1 0; 1 0 1 0 0 0 1; 1 0 1 0 1 0 0; 1 0 0 1 0 0 1; 0 0 1 0 0 1 1; 1 1 0 0 1 0 0; 1 1 0 0 0 0 1; 1 1 0 1 0 0 0; 0 0 0 0 1 1 1; 0 1 0 1 1 0 0; 0 1 0 0 1 1 0; 0 0 0 1 0 1 1; 1 0 0 1 0 1 0; 1 0 0 0 1 0 1; 0 0 1 1 1 0 0; 0 0 1 0 1 0 1
])
F = q.VERTICES_IN_FACETS;
println(T)
println(F)
for kk in T
    if !(kk in F)
        println(kk)
    end
end

20×29 IncidenceMatrix
[1, 2, 4, 5, 7, 8, 11, 12, 13, 15, 18, 21, 23, 24, 26, 28]
[2, 4, 5, 6, 7, 11, 12, 14, 15, 17, 28, 29]
[1, 2, 3, 4, 6, 9, 11, 18, 19, 21, 23, 24]
[1, 2, 4, 6, 7, 11, 12, 14, 15, 18, 19, 21]
[2, 3, 6, 10, 11, 12, 14, 15, 16, 18, 19, 21, 23, 27, 28, 29]
[1, 3, 4, 5, 6, 7, 9, 11, 12, 14, 16, 17, 19, 20, 21, 25, 26]
[3, 5, 6, 10, 11, 12, 14, 16, 17, 25, 28, 29]
[3, 5, 10, 11, 12, 13, 16, 21, 23, 25, 26, 28]
[1, 3, 4, 5, 9, 11, 13, 21, 23, 24, 25, 26]
[2, 8, 10, 13, 15, 18, 22, 23, 24, 27, 28, 29]
[1, 4, 5, 7, 8, 9, 13, 17, 20, 22, 24, 25, 26]
[1, 3, 8, 9, 10, 13, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
[5, 7, 8, 10, 12, 13, 14, 15, 16, 17, 20, 22, 25, 26, 27, 28, 29]
[3, 6, 9, 10, 14, 16, 17, 19, 20, 22, 25, 27, 29]
[1, 2, 4, 6, 7, 8, 9, 14, 15, 17, 18, 19, 20, 22, 24, 27, 29]
[1, 7, 8, 12, 14, 15, 16, 18, 19, 20, 21, 26, 27]
[8, 10, 12, 13, 15, 16, 18, 21, 23, 26, 27, 28]
[2, 4, 5, 7, 8, 13, 15, 17, 22, 24, 28, 29]
[2, 3, 6, 9, 10, 18, 19, 22, 23, 24, 27, 29]
[2,

In [ ]:
# Definiere die Sets
#bigger set
K = [Set([1, 2, 4, 5, 7, 8, 11, 12, 13, 15, 18, 21, 23, 24, 26, 28])
Set([2, 4, 5, 6, 7, 11, 12, 14, 15, 17, 28, 29])
Set([1, 2, 3, 4, 6, 9, 11, 18, 19, 21, 23, 24])
Set([1, 2, 4, 6, 7, 11, 12, 14, 15, 18, 19, 21])
Set([2, 3, 6, 10, 11, 12, 14, 15, 16, 18, 19, 21, 23, 27, 28, 29])
Set([1, 3, 4, 5, 6, 7, 9, 11, 12, 14, 16, 17, 19, 20, 21, 25, 26])#
Set([3, 5, 6, 10, 11, 12, 14, 16, 17, 25, 28, 29])
Set([3, 5, 10, 11, 12, 13, 16, 21, 23, 25, 26, 28])
Set([1, 3, 4, 5, 9, 11, 13, 21, 23, 24, 25, 26])
Set([2, 8, 10, 13, 15, 18, 22, 23, 24, 27, 28, 29])
Set([1, 4, 5, 7, 8, 9, 13, 17, 20, 22, 24, 25, 26])
Set([1, 3, 8, 9, 10, 13, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])
Set([5, 7, 8, 10, 12, 13, 14, 15, 16, 17, 20, 22, 25, 26, 27, 28, 29])
Set([3, 6, 9, 10, 14, 16, 17, 19, 20, 22, 25, 27, 29])
Set([1, 2, 4, 6, 7, 8, 9, 14, 15, 17, 18, 19, 20, 22, 24, 27, 29])
Set([1, 7, 8, 12, 14, 15, 16, 18, 19, 20, 21, 26, 27])
Set([8, 10, 12, 13, 15, 16, 18, 21, 23, 26, 27, 28])
Set([2, 4, 5, 7, 8, 13, 15, 17, 22, 24, 28, 29])
Set([2, 3, 6, 9, 10, 18, 19, 22, 23, 24, 27, 29])
Set([2, 3, 4, 5, 6, 9, 10, 11, 13, 17, 22, 23, 24, 25, 28, 29])]
#smaller set
L = [Set([1, 2, 4, 5, 7, 8, 11, 12, 13, 15, 18, 20, 22, 23, 25, 27])
Set([1, 2, 3, 4, 6, 9, 11, 18, 19, 20, 22, 23])
Set([1, 3, 4, 5, 9, 11, 13, 20, 22, 23, 24, 25])
Set([1, 2, 4, 6, 7, 11, 12, 14, 15, 18, 19, 20])
Set([2, 4, 5, 6, 7, 11, 12, 14, 15, 17, 27, 28])
Set([2, 3, 6, 10, 11, 12, 14, 15, 16, 18, 19, 20, 22, 26, 27, 28])
Set([1, 3, 4, 5, 6, 7, 9, 11, 12, 14, 16, 17, 19, 20, 24, 25])#
Set([3, 5, 10, 11, 12, 13, 16, 20, 22, 24, 25, 27])
Set([3, 5, 6, 10, 11, 12, 14, 16, 17, 24, 27, 28])
Set([1, 2, 4, 6, 7, 8, 9, 14, 15, 17, 18, 19, 21, 23, 26, 28])
Set([2, 4, 5, 7, 8, 13, 15, 17, 21, 23, 27, 28]) #Set([2, 4, 5, 6, 7, 11, 12, 14, 15, 17, 28, 29])
Set([1, 7, 8, 12, 14, 15, 16, 18, 19, 20, 25, 26])#
Set([1, 7, 8, 9, 14, 16, 17, 19, 21, 24, 25, 26])#
Set([2, 3, 6, 9, 10, 18, 19, 21, 22, 23, 26, 28])#
Set([1, 3, 8, 9, 10, 13, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26])
Set([3, 6, 9, 10, 14, 16, 17, 19, 21, 24, 26, 28])
Set([2, 8, 10, 13, 15, 18, 21, 22, 23, 26, 27, 28])
Set([8, 10, 12, 13, 15, 16, 18, 20, 22, 25, 26, 27])
Set([1, 4, 5, 7, 8, 9, 13, 17, 21, 23, 24, 25])
Set([5, 7, 8, 10, 12, 13, 14, 15, 16, 17, 21, 24, 25, 26, 27, 28])
Set([2, 3, 4, 5, 6, 9, 10, 11, 13, 17, 21, 22, 23, 24, 27, 28])]
for kset in L
    liste = []
    for i in 20:29
        if i in kset
            delete!(kset, i)
            push!(liste, i+1)
           
        end
    end
    for j in liste
            push!(kset, j)
        
    end
end
# Funktion, um zu überprüfen, ob ein Set aus L in einem Set aus K enthalten ist
function check_subset(K, L)
    list1 = []
    list2 = []
    for (i, k_set) in enumerate(K)
        for (j, l_set) in enumerate(L)
            if Set(l_set)== Set(k_set)
                push!(list1, i)
                push!(list2, j)
                
            end
        end
    end
    return list1, list2
end

# Beispielaufruf

K2 = [[1, 2, 6], [2, 3, 5], [2, 4, 7], [2, 3, 6], [3, 4, 6], [2, 3, 7], [1, 3, 6], [1, 5, 6], [2, 6, 7], [4, 5, 7], [2, 3, 4], [1, 3, 4], [4, 5, 6], [1, 3, 7], [1, 3, 5], [1, 4, 7], [3, 6, 7], [1, 2, 5], [1, 2, 7], [1, 6, 7], [1, 2, 4], [5, 6, 7], [2, 4, 5], [2, 5, 6], [4, 6, 7], [1, 4, 6], [1, 5, 7], [3, 4, 5], [3, 5, 7]]
L2 = [[1, 2, 6], [2, 3, 5], [2, 4, 7], [2, 3, 6], [3, 4, 6], [2, 3, 7], [1, 3, 6], [1, 5, 6], [2, 6, 7], [4, 5, 7], [2, 3, 4], [1, 3, 4], [4, 5, 6], [1, 3, 7], [1, 3, 5], [1, 4, 7], [3, 6, 7], [1, 2, 5], [1, 2, 7], [1, 2, 4], [5, 6, 7], [2, 4, 5], [2, 5, 6], [4, 6, 7], [1, 4, 6], [1, 5, 7], [3, 4, 5], [3, 5, 7]]

#prints facets which are not facets of K
for r in 1:21
    if !(r in check_subset(K, L)[2])
        println([K2[i] for i in L[r]])
    end
end



7
Set([5, 16, 7, 25, 12, 17, 1, 19, 4, 6, 21, 11, 9, 26, 14, 3])
[[3, 4, 6], [1, 4, 7], [1, 3, 6], [4, 6, 7], [1, 3, 4], [3, 6, 7], [1, 2, 6], [1, 2, 7], [2, 3, 6], [2, 3, 7], [1, 2, 4], [2, 3, 4], [2, 6, 7], [1, 4, 6], [1, 3, 7], [2, 4, 7]]
10
Set([29, 7, 24, 8, 17, 1, 19, 22, 4, 6, 15, 2, 27, 18, 9, 14])
[[3, 5, 7], [1, 3, 6], [2, 5, 6], [1, 5, 6], [3, 6, 7], [1, 2, 6], [1, 2, 7], [5, 6, 7], [2, 3, 6], [2, 3, 7], [1, 3, 5], [2, 3, 5], [1, 5, 7], [1, 2, 5], [2, 6, 7], [1, 3, 7]]
12
Set([16, 7, 12, 8, 1, 19, 15, 21, 27, 18, 26, 14])
[[1, 4, 7], [1, 3, 6], [1, 3, 4], [1, 5, 6], [1, 2, 6], [1, 2, 7], [1, 3, 5], [1, 2, 4], [1, 5, 7], [1, 2, 5], [1, 4, 6], [1, 3, 7]]
13
Set([16, 7, 25, 8, 17, 1, 19, 22, 27, 9, 26, 14])
[[1, 4, 7], [1, 3, 6], [4, 6, 7], [1, 5, 6], [3, 6, 7], [1, 2, 6], [1, 2, 7], [5, 6, 7], [1, 5, 7], [2, 6, 7], [1, 4, 6], [1, 3, 7]]
15
Set([16, 25, 24, 8, 1, 19, 22, 13, 23, 21, 10, 18, 9, 26, 27, 3])
[[1, 4, 7], [4, 6, 7], [2, 5, 6], [1, 5, 6], [1, 2, 6], [1, 2, 7], [5, 6,

In [6]:
#encodes a given subset of 1,2,..n as a number in 1,... (n_elemnts choose r)
function el(b)
    possible_b = collect(combinations(1:n_elements, rk))
    index = -2
    for (l, x) in enumerate(possible_b)
        if x == b
            index = l
        end
    end
    return index
end


el (generic function with 1 method)

In [10]:
subs= [SmallBitSet{UInt64}([])]
for matroid in all_matroids[2:length(all_matroids)]
    subset = SmallBitSet{UInt64}([])
    for basis in bases(matroid)
        a = el(basis)
        b = [a]
        b = SmallBitSet{UInt64}(b)
        subset = union(subset, b)
    end
    push!(subs, subset) # ????
end
#details: add 


In [11]:
hasrank(subs)



5-element Vector{Int64}:
 4518
 1158
   10
   10
    1

In [13]:
function find_min(p, a, b)
    minim = 1
    for (i, z) in enumerate(p)
        if z < a && z < b && subs[i]> subs[minim] && ! (z == a ) && !(z==b) 
            minim = i
        end
    end
    return minim
end

liste = []
#for (i, z) in enumerate(p)
#    if z < p[4518] && z < p[1158] && z > p] 
#        liste.append
#    end
#end
println(find_min(subs,subs[4518], subs[1158]))
M = all_matroids[4728]
println(length(bases(M)))


#M = all_matroids[
println(find_min(subs,subs[4518], subs[1158]))
println(find_min(subs,subs[4518], subs[1158]))
println(is_connected(M))
println(length(bases(M)))
findfirst(z -> subs[4728] < z < subs[1158], subs)
println("length of meet")
println(length(bases(all_matroids[3258])))
println(getrank(subs, subs[4728]))
println(getrank(subs, subs[3258]))
println(find_min(subs, subs[4728], subs[1158]))
println(findfirst(z -> subs[5358] < z < subs[3258], subs))


4728
30
4728
4728
true
30
length of meet
31
9
9
5358
4728


In [ ]:
println(bases(all_matroids[5358]))
println(bases(all_matroids[55]))
println(bases(all_matroids[3258]))
println(getrank(subs, subs[5358]))

[[1, 2, 6], [2, 3, 5], [3, 4, 7], [2, 5, 7], [2, 3, 6], [2, 3, 7], [1, 3, 6], [2, 4, 6], [1, 5, 6], [2, 6, 7], [4, 5, 7], [2, 3, 4], [1, 3, 4], [1, 3, 7], [1, 4, 7], [1, 3, 5], [3, 5, 6], [4, 5, 6], [3, 6, 7], [1, 2, 5], [1, 2, 7], [1, 6, 7], [1, 2, 4], [5, 6, 7], [2, 4, 5], [4, 6, 7], [1, 4, 6], [1, 5, 7], [3, 4, 5]]
[[1, 2, 6], [2, 3, 5], [3, 4, 7], [2, 5, 7], [2, 3, 6], [2, 3, 7], [1, 3, 6], [2, 4, 6], [1, 5, 6], [2, 6, 7], [4, 5, 7], [2, 3, 4], [1, 3, 4], [1, 3, 7], [1, 4, 7], [1, 3, 5], [3, 5, 6], [4, 5, 6], [3, 6, 7], [1, 2, 5], [1, 2, 7], [1, 6, 7], [1, 2, 4], [5, 6, 7], [2, 4, 5], [2, 5, 6], [4, 6, 7], [1, 4, 6], [1, 5, 7], [3, 4, 5]]
[[1, 2, 6], [2, 3, 5], [3, 4, 7], [2, 5, 7], [2, 3, 6], [2, 3, 7], [1, 3, 6], [2, 4, 6], [1, 5, 6], [2, 6, 7], [4, 5, 7], [2, 3, 4], [1, 3, 4], [1, 3, 7], [1, 4, 7], [1, 3, 5], [3, 5, 6], [4, 5, 6], [3, 6, 7], [1, 2, 5], [1, 2, 7], [1, 6, 7], [1, 2, 4], [5, 6, 7], [2, 4, 5], [2, 5, 6], [4, 6, 7], [1, 4, 6], [1, 5, 7], [3, 4, 5], [3, 5, 7]]
8


In [ ]:
t = []
function getchain(p, x, y)
    i = findfirst(z -> x < z < y, p)
    push!(t,i)
    i === nothing ? 1 : getchain(p, x, p[i]) + getchain(p, p[i], y)
end
#getchain(subs, subs[1], subs[3258])
println(find_min(subs, subs[4728], subs[13944]))
println(getrank(subs, subs[17859]))
println(getchain(subs, subs[17859], subs[4098]))
println(t)
#println((subs, subs[4728], subs[32408]))
list_relevant = [17859, 5358, 5568, 4728, 13944, 4098, 3258]
for index in list_relevant
    print("index ")
    println(index)
    println(bases(all_matroids[index]))
    println(is_connected(all_matroids[index-1]))
    println(connected_components(all_matroids[index]))
end
getdist(subs, subs[17859], subs[5358])
println(subs[4098] < subs[3258])

17859
6
2
Any[13944, nothing, nothing]
index 17859
[[1, 3, 7], [1, 2, 6], [3, 4, 7], [3, 6, 7], [1, 2, 7], [2, 4, 7], [2, 3, 6], [1, 6, 7], [1, 2, 4], [2, 3, 7], [1, 3, 6], [4, 6, 7], [1, 4, 6], [2, 4, 6], [2, 3, 4], [1, 3, 4]]
false
[[1, 2, 3, 4, 6, 7], [5]]
index 5358
[[1, 2, 6], [2, 3, 5], [3, 4, 7], [2, 5, 7], [2, 3, 6], [2, 3, 7], [1, 3, 6], [2, 4, 6], [1, 5, 6], [2, 6, 7], [4, 5, 7], [2, 3, 4], [1, 3, 4], [1, 3, 7], [1, 4, 7], [1, 3, 5], [3, 5, 6], [4, 5, 6], [3, 6, 7], [1, 2, 5], [1, 2, 7], [1, 6, 7], [1, 2, 4], [5, 6, 7], [2, 4, 5], [4, 6, 7], [1, 4, 6], [1, 5, 7], [3, 4, 5]]
true
[[1, 2, 3, 4, 5, 6, 7]]
index 5568
[[1, 2, 6], [2, 3, 5], [3, 4, 7], [2, 5, 7], [2, 3, 6], [2, 3, 7], [1, 3, 6], [2, 4, 6], [1, 5, 6], [2, 6, 7], [4, 5, 7], [2, 3, 4], [1, 3, 4], [1, 3, 7], [1, 4, 7], [1, 3, 5], [3, 5, 6], [4, 5, 6], [3, 6, 7], [1, 2, 5], [1, 2, 7], [1, 2, 4], [5, 6, 7], [2, 4, 5], [4, 6, 7], [1, 4, 6], [1, 5, 7], [3, 4, 5]]
true
[[1, 2, 3, 4, 5, 6, 7]]
index 4728
[[1, 2, 6], [2, 3, 5

In [14]:
using Oscar
using Combinatorics
n_elements = 7;
rk = 3;
db = Polymake.Polydb.get_db();
collection = db["Matroids.Small"];
query = Dict("RANK"=>rk,"N_ELEMENTS"=>n_elements, "SIMPLE"=>true);
results = Polymake.Polydb.find(collection, query);
oscar_matroids2 = [Matroid(pm) for pm in results];

In [ ]:
all_matroids2 =Matroid[];
counter =0
ground_set = [i for i in 1:n_elements];
permuts = permutations(ground_set);
for m in oscar_matroids2
    iso= [];
    for perm in permuts
        #println("current permutation is")
        #println(perm)
        current_b = Set();
        #println(current_b);

        for b in bases(m)
            #println("current base is" )
            #println(b)
            permuted_b = Set()
            for e in b
                push!(permuted_b, perm[e])
            end
            push!(current_b, permuted_b);
        end
        current_b = Set([Set(b) for b in current_b])
    if !(current_b in iso)
        push!(iso, current_b);
        counter = counter +1;
            #println(iso)
            #println(iso);
    end
    end 
    for m in iso
        push!(all_matroids2, (matroid_from_bases(m, n_elements)))      
    end
   
end
N = length(all_matroids2)
println(N)

subs2= [SmallBitSet{UInt64}([])]

for matroid in all_matroids2
    subset = SmallBitSet{UInt64}([])
    for basis in bases(matroid)
        a = el(basis)
        b = [a]
        b = SmallBitSet{UInt64}(b)
        subset = union(subset, b)
    end
    push!(subs2, subset) # ????
end

dummy = matroid_from_bases([[]], n_elements)
# insert a dummy to make indices of subs and all_matroids fit. The dummy in subs2 is needed to have an artificial minimum
insert!(all_matroids2, 1, dummy)


8389


In [31]:
hasrank(subs2)


5-element Vector{Int64}:
 4518
 1158
    4
    3
    1

In [81]:
function getchain(p, x, y)
    t = []
    getchain_h(p, x, y, t)
    return t
end

function getchain_h(p, x, y, t)
    i = findfirst(z -> x < z < y, p)
    push!(t,i)
    i === nothing ? 1 : getchain_h(p, x, p[i], t) + getchain_h(p, p[i], y, t)
end

function find_min2(p, a, b)
    minim = 1
    for (i, z) in enumerate(p)
        if z < a && z < b && p[i]> p[minim] && ! (z == a ) && !(z==b) 
            minim = i
        end
    end
    return minim
end

println(getchain(subs2, subs2[8038], subs2[4518]))
#find_min2(subs2, subs2[4098], subs2[4728])
println(find_min2(subs2, subs2[4518], subs2[1998]))
println(subs2[1998]> subs2[5568])
println(getrank(subs2, subs2[8038]))
println(bases(all_matroids2[5358]))
println(bases(all_matroids2[5568]))

liste = []
for m in subs
    if subs2[1998]> m
        push!(liste, m)
    end
end

println(length(liste))



Any[nothing]
8038
false
1
[[1, 2, 6], [2, 3, 5], [2, 4, 7], [2, 3, 6], [3, 4, 6], [2, 3, 7], [1, 3, 6], [1, 5, 6], [2, 6, 7], [4, 5, 7], [2, 3, 4], [1, 3, 4], [4, 5, 6], [1, 3, 7], [1, 3, 5], [1, 4, 7], [3, 6, 7], [1, 2, 5], [1, 2, 7], [1, 6, 7], [1, 2, 4], [5, 6, 7], [2, 4, 5], [2, 5, 6], [4, 6, 7], [1, 4, 6], [1, 5, 7], [3, 4, 5], [3, 5, 7]]
[[1, 2, 6], [2, 3, 5], [2, 4, 7], [2, 3, 6], [3, 4, 6], [2, 3, 7], [1, 3, 6], [1, 5, 6], [2, 6, 7], [4, 5, 7], [2, 3, 4], [1, 3, 4], [4, 5, 6], [1, 3, 7], [1, 3, 5], [1, 4, 7], [3, 6, 7], [1, 2, 5], [1, 2, 7], [1, 2, 4], [5, 6, 7], [2, 4, 5], [2, 5, 6], [4, 6, 7], [1, 4, 6], [1, 5, 7], [3, 4, 5], [3, 5, 7]]
2568


In [ ]:
#get all minimal matroids 
minimals = []
for (i, m) in enumerate(subs2)
    if getrank(subs2 ,m )== 1
        push!(minimals, i)
    end
end

minimals_con = []

for i in minimals
    m = all_matroids2[i]
    if is_connected(m)
        push!(minimals_con,i) 
    end
end
print(length(minimals_con))


1465

In [ ]:
for (index, o) in enumerate(all_matroids2)
    if is_subset(independent_sets(o), independent_sets(all_matroids2[4488])) 
        println(index)
    end
end

1
4488


In [ ]:
M = all_matroids2[4488]
print(bases(all_matroids2[4488]))


[[1, 2, 6], [3, 4, 7], [2, 3, 5], [2, 4, 7], [2, 3, 6], [2, 5, 7], [3, 4, 6], [2, 3, 7], [1, 3, 6], [2, 6, 7], [4, 5, 7], [2, 3, 4], [1, 3, 4], [1, 3, 7], [4, 5, 6], [3, 5, 6], [1, 4, 5], [1, 3, 5], [3, 6, 7], [1, 2, 7], [1, 2, 5], [1, 6, 7], [1, 2, 4], [5, 6, 7], [2, 4, 5], [2, 5, 6], [4, 6, 7], [1, 4, 6], [1, 5, 7], [3, 4, 5]]

LoadError: UndefVarError: `matroids` not defined in `Main`
Suggestion: check for spelling errors or missing imports.